I used this notebook to help write a recommender function step by step. This function can be found in the Recommender class in Recommender.py. Mapper added to tags is not implemented here.

In [98]:
import numpy as np
from numpy.linalg import norm
import pandas as pd

compare_song = '295cc'
difficulty = 4

In [99]:
songs = pd.read_csv('datasets/beatsaversongs.csv', delimiter=',')
songs.BeatMapId = songs.BeatMapId.astype('string')
songs.UploaderName = songs.UploaderName.astype('category')
songs.IsRanked = songs.IsRanked.astype(int)
songs.IsCurated = songs.IsCurated.astype(int)
songs.HasChroma = songs.HasChroma.astype(int)
songs.HasCinema = songs.HasCinema.astype(int)
songs.HasMappingExtensions = songs.HasMappingExtensions.astype(int)
songs.HasNoodleExtensions = songs.HasNoodleExtensions.astype(int)

def add_uploader_to_tags(tags, uploader):
    if len(tags) > 0 and tags[0] != 'nan':
        tags.append(uploader)
    return tags

songs.Tags = songs.apply(lambda r: add_uploader_to_tags(str(r.Tags).split(';'), r.UploaderName), axis=1)
songs.Tags = songs.Tags.apply(lambda r: ';'.join(r))
songs.Tags = songs.Tags.astype('string')

In [100]:
songs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115783 entries, 0 to 115782
Data columns (total 25 columns):
 #   Column                Non-Null Count   Dtype   
---  ------                --------------   -----   
 0   BeatMapId             115783 non-null  string  
 1   UploaderName          115783 non-null  category
 2   Duration              115783 non-null  int64   
 3   Bpm                   115783 non-null  float64 
 4   IsRanked              115783 non-null  int32   
 5   IsCurated             115783 non-null  int32   
 6   Tags                  115783 non-null  string  
 7   CommunityRating       115783 non-null  float64 
 8   Bombs                 115783 non-null  int64   
 9   Characteristic        115783 non-null  int64   
 10  HasChroma             115783 non-null  int32   
 11  HasCinema             115783 non-null  int32   
 12  Difficulty            115783 non-null  int64   
 13  Events                115783 non-null  int64   
 14  HasMappingExtensions  115783 non-nul

In [101]:
songs['Tags'].explode().value_counts()

nan                               68578
anime;j-pop;Joetastic               172
balanced;anime;j-pop;Joetastic      120
k-pop;simplymarvellous              103
balanced;k-pop;eurnkung              92
                                  ...  
dance;clownglitz                      1
alternative;indie;darkrailord3        1
pop;comedy-meme;elijoria              1
video-game-soundtrack;infotron        1
anime;dance;datkami                   1
Name: Tags, Length: 19419, dtype: Int64

Dealing with outliers. Not deleting but replacing them with an "A lot" equivalent to keep the items relevant as someone might be interested in these outliers.

In [102]:
songs.loc[songs.Njs > 40, "Njs"] = 40
songs.loc[songs.Njs < -10, "Njs"] = -10
songs.loc[songs.Offset > 20, "Offset"] = 20
songs.loc[songs.Offset < -20, "Offset"] = -20
songs.loc[songs.Notes > 10000, 'Notes'] = 10000
songs.loc[songs.Events > 20000, 'Events'] = 20000
songs.loc[songs.Bombs > 5000, 'Bombs'] = 5000
songs.loc[songs.Obstacles > 10000, 'Obstacles'] = 10000
songs.loc[songs.Errors > 20, 'Errors'] = 20
songs.loc[songs.Resets > 20, 'Resets'] = 20
songs.loc[songs.Warns > 20, 'Warns'] = 20
songs.loc[songs.Bpm > 1000, 'Bpm'] = 1000
songs.loc[songs.Duration > 1800, 'Duration'] = 1800
songs

,BeatMapId,UploaderName,Duration,Bpm,IsRanked,IsCurated,Tags,CommunityRating,Bombs,Characteristic,...,HasNoodleExtensions,Njs,Notes,Nps,Obstacles,Offset,Stars,Errors,Resets,Warns
0,2d675,Joetastic,91,140.0,0,0,balanced;anime;j-pop;Joetastic,0.5000,0,0,...,0,12.0,269,3.062,65,0.000,0.00,0,0,0
1,2d675,Joetastic,91,140.0,0,0,balanced;anime;j-pop;Joetastic,0.5000,0,0,...,0,15.0,423,4.815,65,-0.250,0.00,0,0,0
2,2d675,Joetastic,91,140.0,0,0,balanced;anime;j-pop;Joetastic,0.5000,0,0,...,0,18.0,489,5.566,65,-0.600,0.00,0,0,0
3,2d674,damedame,234,144.0,0,0,accuracy;balanced;anime;j-pop;damedame,0.5942,0,0,...,0,18.0,1046,4.768,16,-0.667,0.00,0,0,0
4,2d673,Gabriel,180,130.0,0,0,tech;balanced;electronic;Gabriel,0.6408,47,0,...,0,16.0,856,5.040,145,-0.580,0.00,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115778,4,jugglernaut,117,200.0,0,0,electronic;jugglernaut,0.2801,0,0,...,0,10.0,233,2.287,0,0.000,0.00,0,0,0
115779,4,jugglernaut,117,200.0,0,0,electronic;jugglernaut,0.2801,0,0,...,0,10.0,365,3.479,0,0.000,0.00,0,0,0
115780,4,jugglernaut,117,200.0,0,0,electronic;jugglernaut,0.2801,0,0,...,0,10.0,490,4.670,0,0.000,0.00,0,0,0
115781,2,jugglernaut,91,179.0,0,0,anime;jugglernaut,0.8163,0,0,...,0,10.0,336,3.962,0,0.000,0.00,20,0,20


In [103]:

compare_tags = songs.loc[songs.BeatMapId == compare_song].loc[songs.Difficulty == difficulty].Tags.unique()[0].split(
    ';')
song_sims = songs
non_meta_columns = ['BeatMapId', 'UploaderName', 'Tags', 'TagSim']

In [104]:
def jaccard_similarity(a, b) -> float:
    a = set(a)
    b = set(b)
    j = float(len(a.intersection(b))) / len(a.union(b))
    return j

In [105]:
tag_sims = [jaccard_similarity(compare_tags, tags.split(';')) for tags in songs.Tags]
song_sims['TagSim'] = tag_sims
vals = list(song_sims.TagSim.value_counts().index)
vals.sort(reverse=True)

In [106]:
compare_meta = song_sims.loc[song_sims.BeatMapId == compare_song].loc[song_sims.Difficulty == difficulty].drop(
    columns=non_meta_columns)

song_metadata = song_sims.loc[song_sims.TagSim.isin(vals[:2])].drop(columns=non_meta_columns)
song_metadata = (song_metadata - song_metadata.min()) / (song_metadata.max() - song_metadata.min())
song_metadata = song_metadata.fillna(0)
song_metadata.Difficulty = song_metadata.Difficulty * 5
song_metadata

,Duration,Bpm,IsRanked,IsCurated,CommunityRating,Bombs,Characteristic,HasChroma,HasCinema,Difficulty,...,HasNoodleExtensions,Njs,Notes,Nps,Obstacles,Offset,Stars,Errors,Resets,Warns
9818,0.000000,1.0,1.0,1.0,0.00000,0.193476,0.0,1.0,0.0,0.00,...,0.0,0.000000,0.000000,0.000000,0.682731,0.15625,0.184438,0.0,0.60,0.00
9819,0.000000,1.0,1.0,1.0,0.00000,0.290214,0.0,1.0,0.0,1.25,...,0.0,0.285714,0.065800,0.174488,0.682731,1.00000,0.231508,0.0,0.35,0.00
9820,0.000000,1.0,1.0,1.0,0.00000,0.672666,0.0,1.0,0.0,2.50,...,0.0,0.571429,0.094874,0.251495,0.441767,0.68750,0.311239,0.0,0.60,0.00
9821,0.000000,1.0,1.0,1.0,0.00000,0.865017,0.0,1.0,0.0,3.75,...,0.0,0.714286,0.221882,0.588150,0.273092,0.50000,0.666667,0.1,0.70,0.60
9822,0.000000,1.0,1.0,1.0,0.00000,1.000000,0.0,1.0,0.0,5.00,...,0.0,0.857143,0.377200,1.000000,0.220884,0.31250,1.000000,1.0,0.90,1.00
18881,0.314516,0.0,0.0,0.0,0.80084,0.000000,0.0,0.0,0.0,5.00,...,0.0,0.428571,0.418516,0.617684,1.000000,0.43750,0.000000,0.0,0.00,1.00
23103,1.000000,1.0,0.0,1.0,1.00000,0.000000,0.0,0.0,0.0,0.00,...,0.0,0.000000,0.312165,0.059250,0.000000,0.00000,0.000000,0.0,0.00,0.00
23104,1.000000,1.0,0.0,1.0,1.00000,0.013498,0.0,0.0,0.0,1.25,...,0.0,0.285714,0.475899,0.240623,0.160643,0.93750,0.000000,0.0,0.15,0.00
23105,1.000000,1.0,0.0,1.0,1.00000,0.021372,0.0,0.0,0.0,2.50,...,0.0,0.714286,0.749044,0.543396,0.975904,0.56250,0.000000,0.0,0.15,0.65
23106,1.000000,1.0,0.0,1.0,1.00000,0.113611,0.0,0.0,0.0,3.75,...,0.0,0.857143,0.882938,0.691792,0.176707,0.40625,0.000000,0.0,1.00,1.00


In [107]:
def cosine_sim(a, b):
    a = a.values[0]
    return np.dot(a, b) / (norm(a) * norm(b))


metadata_sims = [cosine_sim(compare_meta, row) for row in song_metadata.values]

In [108]:
song_metadata['MetaSim'] = metadata_sims
song_metadata.sort_values(by='MetaSim').tail(100)

,Duration,Bpm,IsRanked,IsCurated,CommunityRating,Bombs,Characteristic,HasChroma,HasCinema,Difficulty,...,Njs,Notes,Nps,Obstacles,Offset,Stars,Errors,Resets,Warns,MetaSim
18881,0.314516,0.0,0.0,0.0,0.80084,0.000000,0.0,0.0,0.0,5.00,...,0.428571,0.418516,0.617684,1.000000,0.43750,0.000000,0.0,0.00,1.00,0.004155
23107,1.000000,1.0,0.0,1.0,1.00000,0.230596,0.0,0.0,0.0,5.00,...,1.000000,1.000000,0.821526,0.341365,0.31250,0.000000,0.0,1.00,1.00,0.050855
23106,1.000000,1.0,0.0,1.0,1.00000,0.113611,0.0,0.0,0.0,3.75,...,0.857143,0.882938,0.691792,0.176707,0.40625,0.000000,0.0,1.00,1.00,0.060568
23105,1.000000,1.0,0.0,1.0,1.00000,0.021372,0.0,0.0,0.0,2.50,...,0.714286,0.749044,0.543396,0.975904,0.56250,0.000000,0.0,0.15,0.65,0.075923
23104,1.000000,1.0,0.0,1.0,1.00000,0.013498,0.0,0.0,0.0,1.25,...,0.285714,0.475899,0.240623,0.160643,0.93750,0.000000,0.0,0.15,0.00,0.100329
23103,1.000000,1.0,0.0,1.0,1.00000,0.000000,0.0,0.0,0.0,0.00,...,0.000000,0.312165,0.059250,0.000000,0.00000,0.000000,0.0,0.00,0.00,0.125587
9822,0.000000,1.0,1.0,1.0,0.00000,1.000000,0.0,1.0,0.0,5.00,...,0.857143,0.377200,1.000000,0.220884,0.31250,1.000000,1.0,0.90,1.00,0.176593
9821,0.000000,1.0,1.0,1.0,0.00000,0.865017,0.0,1.0,0.0,3.75,...,0.714286,0.221882,0.588150,0.273092,0.50000,0.666667,0.1,0.70,0.60,0.223817
9820,0.000000,1.0,1.0,1.0,0.00000,0.672666,0.0,1.0,0.0,2.50,...,0.571429,0.094874,0.251495,0.441767,0.68750,0.311239,0.0,0.60,0.00,0.287004
9819,0.000000,1.0,1.0,1.0,0.00000,0.290214,0.0,1.0,0.0,1.25,...,0.285714,0.065800,0.174488,0.682731,1.00000,0.231508,0.0,0.35,0.00,0.353744
